# ECSN Test

In [1]:
from StarKiller.initialization import starkiller_initialize
from StarKiller.interfaces import EosType
from StarKiller.interfaces import BurnType
from StarKiller.eos import Eos
from StarKiller.network import Network
import numpy as np

In [2]:
probin_file = "probin_ecsn"

In [3]:
starkiller_initialize(probin_file)

In [4]:
helmholtz = Eos()
ecsn = Network()

In [5]:
def get_eps_nuc_eos(burn_state):
    ecsn.rhs(burn_state)
    print(burn_state.state)
    
    eos_state = burn_state.to_eos_type()
    helmholtz.evaluate(eos_state.eos_input_rt, eos_state)
    print(eos_state.state)
    
    Hnuc = burn_state.state.ydot[ecsn.net_ienuc]
    print("Energy generation rate: {:0.09e}".format(Hnuc))
    
    aion = ecsn.ActualNetworkModule.aion
    zion = ecsn.ActualNetworkModule.zion

    # dX/dt = dY/dt * Aion
    omegadot = burn_state.state.ydot[:ecsn.nspec_evolve] * aion
    print("omegadot = {}".format(omegadot))
    
    cp = eos_state.state.cp
    print("cp = {:0.05e}".format(cp))
    
    dpdt = eos_state.state.dpdt
    print("dpdt = {:0.05e}".format(dpdt))
    
    dpdx = eos_state.state.dpdx
    print("dpdx = {}".format(dpdx))
    
    dhdx = eos_state.state.dhdx
    print("dhdx = {}".format(dhdx))

    print("cp * dpdx/dpdt = {}".format(cp * dpdx/dpdt))
    print("cp * dpdx/dpdt * wdot = {:0.09e}".format(np.dot(cp * dpdx/dpdt, omegadot)))
    print("dhdx = {}".format(dhdx))
    print("dhdx * wdot = {:0.09e}".format(np.dot(dhdx, omegadot)))

    dHcompdx = (cp * dpdx/dpdt - dhdx)
    print("dHcompdx = {}".format(dHcompdx))
    
    Hcomp = np.dot(dHcompdx, omegadot)
    print("Composition energy generation: {:0.09e}".format(Hcomp))

    dedx = eos_state.state.dedx
    print("dedx = {}".format(dedx))
    
    de = -np.dot(dedx, omegadot)
    print("eps_nuc_eos = dedx * wdot (cf. MESA) = {:0.09e}".format(de))

    dabardX = (eos_state.state.abar/aion) * (aion - eos_state.state.abar)
    dzbardX = (eos_state.state.abar/aion) * (zion - eos_state.state.zbar)

    dabardt = np.dot(dabardX, omegadot)
    dzbardt = np.dot(dzbardX, omegadot)

    print("dabar/dt = {:0.09e}".format(dabardt))
    print("dzbar/dt = {:0.09e}".format(dzbardt))

    print("sum omegadot = {:0.16e}".format(np.sum(omegadot)))
    print("sum X = {:0.16e}".format(np.sum(burn_state.state.xn)))
    
    Htot = Hnuc + Hcomp
    print("Hnuc + Hcomp (Maestro): {:0.9e}".format(Htot))

## Test 1 from Josiah

In [6]:
burn_state_1 = BurnType()

burn_state_1.state.rho = 7607783959.50421e0
burn_state_1.state.t = 983559309.749412e0
burn_state_1.state.xn[ecsn.species_map["h1"]] = 0.0
burn_state_1.state.xn[ecsn.species_map["he4"]] = 1.14998095787427E-18
burn_state_1.state.xn[ecsn.species_map["o16"]] = 0.5955320774
burn_state_1.state.xn[ecsn.species_map["o20"]] = 0.203902681
burn_state_1.state.xn[ecsn.species_map["f20"]] = 1.17278643621995E-07
burn_state_1.state.xn[ecsn.species_map["ne20"]] = 0.19753136
burn_state_1.state.xn[ecsn.species_map["mg24"]] = 0.0
burn_state_1.state.xn[ecsn.species_map["al27"]] = 0.0
burn_state_1.state.xn[ecsn.species_map["si28"]] = 0.0027259815
burn_state_1.state.xn[ecsn.species_map["p31"]] = 0.0
burn_state_1.state.xn[ecsn.species_map["s32"]] = 0.0

In [7]:
get_eps_nuc_eos(burn_state_1)

<burn_t>{
    rho : 7607783959.50421,
    t : 983559309.749412,
    e : 2.490743133790312e+18,
    xn : array([0.00000000e+00, 1.14998096e-18, 5.95532077e-01, 2.03902681e-01,
       1.17278644e-07, 1.97531360e-01, 0.00000000e+00, 0.00000000e+00,
       2.72598150e-03, 0.00000000e+00, 0.00000000e+00]),
    cv : 14444247.553061701,
    cp : 14486626.878325198,
    y_e : 0.47945583462538965,
    eta : 87.75746182063078,
    cs : 1099451413.7881703,
    dx : 0.0,
    abar : 17.42469332287015,
    zbar : 8.354370880208164,
    t_old : 6.91452328486113e-310,
    dcvdt : 4.6613249073456e-310,
    dcpdt : 4.66132490734757e-310,
    ydot : array([ 1.16695205e-11, -7.26850364e-12, -4.11345965e-11,  5.79382894e-11,
        7.42255606e-12, -5.81959032e-11,  2.16734518e-12,  0.00000000e+00,
        4.23112889e-12,  1.16695205e-11,  5.05156388e-16, -3.95207429e+01,
       -5.72522257e+08]),
    jac : array([[4.66132491e-310, 4.66132491e-310, 5.30498948e-313,
        0.00000000e+000, 6.91451145e-310,

## Test 2 from Josiah

In [8]:
burn_state_2 = BurnType()

burn_state_2.state.rho = 8346403698.95185e0
burn_state_2.state.t = 656536305.140964e0
burn_state_2.state.xn[ecsn.species_map["h1"]] = 0.0
burn_state_2.state.xn[ecsn.species_map["he4"]] = 8.89957317703129E-23
burn_state_2.state.xn[ecsn.species_map["o16"]] = 0.5999942706e0
burn_state_2.state.xn[ecsn.species_map["o20"]] = 0.3926652894e0
burn_state_2.state.xn[ecsn.species_map["f20"]] = 0.000000002e0
burn_state_2.state.xn[ecsn.species_map["ne20"]] = 0.0073364571e0
burn_state_2.state.xn[ecsn.species_map["mg24"]] = 0.0
burn_state_2.state.xn[ecsn.species_map["al27"]] = 0.0
burn_state_2.state.xn[ecsn.species_map["si28"]] = 3.518236231242E-06
burn_state_2.state.xn[ecsn.species_map["p31"]] = 0.0
burn_state_2.state.xn[ecsn.species_map["s32"]] = 0.0

In [9]:
get_eps_nuc_eos(burn_state_2)

<burn_t>{
    rho : 8346403698.95185,
    t : 656536305.140964,
    e : 2.4366503017450455e+18,
    xn : array([0.00000000e+00, 8.89957318e-23, 5.99994271e-01, 3.92665289e-01,
       2.00000000e-09, 7.33645710e-03, 0.00000000e+00, 0.00000000e+00,
       3.51823623e-06, 0.00000000e+00, 0.00000000e+00]),
    cv : 13444158.983137527,
    cp : 13470518.44019436,
    y_e : 0.46073323962811563,
    eta : 133.98527882572262,
    cs : 1086123519.6557314,
    dx : 0.0,
    abar : 17.391348207658748,
    zbar : 8.012772201215236,
    t_old : 6.91452328486113e-310,
    dcvdt : 4.66132490582783e-310,
    dcpdt : 4.6613249058298e-310,
    ydot : array([ 3.75545872e-17, -2.13349498e-17, -1.35886276e-16,  1.24835159e-12,
        1.35222972e-13, -1.38354101e-12,  5.60711361e-19,  0.00000000e+00,
        1.33342876e-17,  3.75545872e-17,  4.90756421e-25,  3.57280937e-62,
       -1.61389226e+07]),
    jac : array([[4.66132491e-310, 4.66132491e-310, 5.30498948e-313,
        0.00000000e+000, 6.91451145e-31